In [11]:
from flask import Flask, render_template, request, redirect, url_for, session #Needed for web server
from flask_basicauth import BasicAuth

app = Flask(__name__)


from flask_sqlalchemy import SQLAlchemy #Needed for web server
import csv  #Needed for reading/saving the data
import os  #Needed for reading/saving the data
import random #Needed for randomizing the order of the questions


In [12]:
app = Flask(__name__)
app.secret_key = os.urandom(24)
app.config['BASIC_AUTH_USERNAME'] = 'admin'
app.config['BASIC_AUTH_PASSWORD'] = 'admin'

basic_auth = BasicAuth(app)



# Environment variables

NUMBER_OF_QUESTIONS_PER_VALUE  = 1
SURVEY_INTRODUCTION_TEXT = "Welcome to the survey! Please answer the following questions to the best of your ability"


In [13]:
import csv
import random

def load_questions_from_csv(file_path):

   
    
    # Initialize an empty list to store the questions
    questions = []

    # Initialize a set to store unique main values
    main_values = set()

    # Open the CSV file
    with open(file_path, 'r') as file:
        # Create a CSV reader object with '@' as the delimiter
        # The CSV should have the following columns:
        # - questionID
        # - main_value
        # - sub_value
        # - story
        reader = csv.DictReader(file, delimiter="@")
        
        # Iterate over each row in the CSV file
        for row in reader:
            # Append the row (question) to the list of questions
            questions.append(row)
            
            # Add the main value of the row to the set of main values
            main_values.add(row['main_value'])

    # Randomly select two questions for each main value
    selected_questions = []
    
    # Iterate over each main value
    for value in main_values:
        # Filter the questions based on the current main value
        value_questions = [q for q in questions if q['main_value'] == value]
        
        # Randomly select one question from the filtered questions
        selected = random.sample(value_questions, NUMBER_OF_QUESTIONS_PER_VALUE)
        
        # Add the selected question to the list of selected questions
        selected_questions.extend(selected)

    # Return the list of selected questions
    return selected_questions


In [14]:
def save_survey_results_to_csv(file_path, results):
    fieldnames = ['question_id', 'conservation', 'self_transcendence', 'openness_to_change', 'self_enhancement']

    # Check if the file already exists
    file_exists = os.path.isfile(file_path)

    with open(file_path, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write the header only if the file doesn't exist
        if not file_exists:
            writer.writeheader()

        writer.writerows(results)

Below is the code for the survey, in this version 4 values are tracked and recorded for the survey
    Conservation
    Openness to change
    Self Enhancement
    Self Transcendence

    To change the values the templates/survey.html page must also be changed

In [15]:
@app.route('/admin')
@basic_auth.required
def secret_view():
    return render_template('admin.html')


In [16]:
@app.route('/survey', methods=['GET', 'POST'])
def survey():
    if 'current_question' not in session:
        session['current_question'] = 0

    if 'survey_results' not in session:
        session['survey_results'] = []

    questions = load_questions_from_csv('question_bank/questions.csv')
    current_question = session['current_question']
    survey_results = session['survey_results']

    if request.method == 'POST':
        # Process the submitted survey form and store the ratings
        question_id = request.form['question_id']
        conservation_rating = request.form['conservation']
        self_transcendence_rating = request.form['self_transcendence']
        openness_to_change_rating = request.form['openness_to_change']
        self_enhancement_rating = request.form['self_enhancement']

        # Store the question_id and rating as desired
        survey_results.append({
            'question_id': question_id,
            'conservation': conservation_rating,
            'self_transcendence': self_transcendence_rating,
            'openness_to_change': openness_to_change_rating,
            'self_enhancement': self_enhancement_rating
        })

        if current_question == len(questions) - 1:
            # If the last question has been answered, show the thank you page
            save_survey_results_to_csv('survey_results.csv', survey_results)
            session.pop('survey_results')  # Remove survey results from session
            return redirect(url_for('thank_you'))
        else:
            # After processing the submission, increment the current question
            session['current_question'] += 1
            return redirect(url_for('survey'))

    else:
        if current_question < 0:
            current_question = 0
        elif current_question >= len(questions):
            # If the current question exceeds the question count, redirect to the last question
            return redirect(url_for('survey'))

        question = questions[current_question]
        print(question) 

        # Render the survey page with the current question
        return render_template('survey.html', random_question=question)

In [17]:
@app.route('/', methods=['GET', 'POST'])
def index():

    session['survey_text'] = SURVEY_INTRODUCTION_TEXT
    
    #If they click the button to start the survey, then redirect them to the survey page
    
    if request.method == 'POST':
        return redirect(url_for('survey'))
    

    return render_template('index.html')

In [18]:
@app.route('/thank_you')
def thank_you():
    return render_template('thank_you.html', survey_results=session.get('survey_results', []))


In [19]:
@app.route('/admin', methods=['GET', 'POST'])
def admin_panel():
    if request.method == 'POST':
        # Process the admin form submission
        # You can add your own logic here to handle admin actions
        return redirect(url_for('admin_panel'))
    else:
        # Render the admin panel template
        return render_template('admin_panel.html')


In [20]:
if __name__ == "__main__":
  app.debug = False
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jul/2023 14:19:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2023 14:19:07] "GET /survey HTTP/1.1" 200 -


{'id': '5', 'main_value': 'conservation', 'sub_value': 'Security-Societal', 'story': "The office of salesperson John Smith was bustling with activity. He had been in the business for nearly a decade and was well known for his commitment to the security and stability of his country. He had grown up in a small town, where the people were proud of their country and its ideals. John was passionate about his job and made sure that he was always up to date on the latest trends in the market. He was constantly looking for ways to increase sales and provide better customer service. He worked diligently to ensure that his clients were satisfied with their purchases and that their money was well spent.John was a firm believer in the importance of his country's security and stability. He was well aware of the threats that existed and was always on the lookout for any potential issues that could affect the safety of his customers. He was also a strong advocate for the protection of the environment

127.0.0.1 - - [07/Jul/2023 14:19:10] "POST /survey HTTP/1.1" 302 -
127.0.0.1 - - [07/Jul/2023 14:19:10] "GET /survey HTTP/1.1" 200 -


{'id': '12', 'main_value': 'openness_to_change', 'sub_value': 'Stimulation', 'story': "John had been working in the sales office for over three years. He had started out as a junior salesperson, but had worked his way up to be one of the top performers in the office. He was always looking for new ways to make sales, and he was always looking for different things to do. He felt that it was important to stay ahead of the curve and to be able to adapt to changing markets. He was always looking for new ideas and ways to make sales. John had a lot of success in the office. He was always looking for new ways to make sales and he was always willing to try new things. He was always willing to take risks and to experiment with different strategies. He was always looking for ways to improve and to make the most of the opportunities that were presented to him. John's colleagues respected him for his willingness to try new things. They knew that he was always looking for different things to do and

127.0.0.1 - - [07/Jul/2023 14:19:14] "POST /survey HTTP/1.1" 302 -
127.0.0.1 - - [07/Jul/2023 14:19:14] "GET /survey HTTP/1.1" 200 -


{'id': '8', 'main_value': 'self_enhancement', 'sub_value': 'Power-Dominance', 'story': "John had been working in the sales office for almost three years now. He had started off as a junior salesperson, but had quickly worked his way up to a senior position. He believed that if he was in charge of the office, he could ensure that everything ran smoothly and efficiently. He was also confident that his leadership would lead to greater success and higher profits. John's approach to managing the office was to be strict and demanding. He expected all of his employees to follow his rules and to adhere to his expectations. If anyone failed to comply, they were quickly reprimanded and reminded of their place. John was also known for being quite aggressive when it came to selling. He was not afraid to push his clients to buy more and to offer discounts to close the deal. He was confident that this approach would lead to higher sales figures and greater profits. However, John's approach to managi

127.0.0.1 - - [07/Jul/2023 14:19:19] "POST /survey HTTP/1.1" 302 -
127.0.0.1 - - [07/Jul/2023 14:19:19] "GET /survey HTTP/1.1" 200 -


{'id': '2', 'main_value': 'self_transcendence', 'sub_value': 'Universilism-Nature', 'story': "The sales office was buzzing with activity as the team prepared for the upcoming quarter. Among the salespeople, there was one who stood out. Her name was Jennifer, and she was a passionate advocate for defending nature. Jennifer was deeply committed to doing her part to protect the environment, and that was reflected in her work. Whenever she had the opportunity, she took part in activities to defend nature. She volunteered for beach cleanups, attended rallies, and even organized her own campaigns to raise awareness about the importance of preserving natural resources. At the office, she was a role model for her colleagues. She encouraged them to reduce their carbon footprint by taking public transportation, recycling, and using reusable containers for lunches. She also encouraged them to participate in activities to defend nature. She organized trips to the beach to clean up plastic and debr

127.0.0.1 - - [07/Jul/2023 14:19:23] "POST /survey HTTP/1.1" 302 -
127.0.0.1 - - [07/Jul/2023 14:19:23] "GET /thank_you HTTP/1.1" 200 -
